In [1]:
import csv
import requests
import os
from datetime import datetime, timedelta
from ajna_commons.utils.sanitiza import sanitizar, mongo_sanitizar

with open(os.path.join('..', 'integracao', 'dte.info')) as dte_info:
    linha = dte_info.readline()
DTE_USERNAME = os.environ.get('DTE_USERNAME')
DTE_PASSWORD = os.environ.get('DTE_PASSWORD')
DTE_USERNAME = linha.split(',')[0]
DTE_PASSWORD = linha.split(',')[1]


In [2]:
with open(os.path.join('..', 'integracao', 'recintos.csv'), encoding='utf-8') as csv_in:
    reader = csv.reader(csv_in)
    recintos_list = [row for row in reader]

In [3]:
DTE_URL = 'https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem'
def get_token_dte(username=DTE_USERNAME, password=DTE_PASSWORD):
    data = {'username': username, 'password': password, 'grant_type': 'password'}
    r = requests.post(DTE_URL + '/token', data=data)
    token = r.json().get('access_token')
    return token

In [4]:
def get_pesagens_dte(datainicial, datafinal, recinto, token):
    payload = {'data_inicio': datetime.strftime(datainicial, '%Y-%m-%d'),
               'data_fim': datetime.strftime(datafinal, '%Y-%m-%d'),
               'cod_recinto': recinto}
    headers = {'Authorization': 'Bearer ' + token}
    r = requests.get(DTE_URL, headers=headers, params=payload)
    print(r.url)
    try:
        lista_pesagens = r.json()['JUP_WS']['Pesagens']['Lista_Pesagens']
    except:
        print(r, r.text)
    return lista_pesagens

In [5]:
from collections import defaultdict
token = get_token_dte()
start = end = datetime.now() - timedelta(days=1)
pesagens_recintos = defaultdict(list)
for linha in recintos_list[1:]:
    recinto = linha[0]
    pesagens_recinto = get_pesagens_dte(start, end, recinto, token)
    if pesagens_recinto and len(pesagens_recinto) > 0:
        print(recinto, len(pesagens_recinto))
        pesagens_recintos[recinto].extend(pesagens_recinto)

https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=25
25 42
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=105
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=22
22 8
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=846
846 75
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=24
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=839
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/api/Pesagem?data_inicio=2019-03-24&data_fim=2019-03-24&cod_recinto=45
https://www.janelaunicaportuaria.org.br/ws_homologacao/sepes/a

In [6]:
def trata_pesagem_dte(registro):
    new_dict = {}
    for key, value in registro.items():
        key = sanitizar(key, mongo_sanitizar)
        value = sanitizar(value, mongo_sanitizar)
        new_dict[key] = value
    new_dict['datahoraentradaiso'] = datetime.strptime(new_dict['datahoraentrada'], '%Y-%m-%d %H:%M:%S')
    datahorasaida = new_dict['datahorasaida']
    if datahorasaida:
        new_dict['datahorasaidaiso'] = datetime.strptime(new_dict['datahorasaida'], '%Y-%m-%d %H:%M:%S')
    new_dict['pesoentradafloat'] = float(new_dict['pesoentrada'].replace(',', '.'))
    new_dict['pesosaidafloat'] = float(new_dict['pesosaida'].replace(',', '.'))
    new_dict['veiculocarregadosaidabool'] = new_dict['veiculocarregadosaida'] == "sim"
    new_dict['veiculocarregadoentradabool'] = new_dict['veiculocarregadoentrada'] == "sim"
    return(new_dict)

i = 0
for recinto, pesagens in pesagens_recintos.items():
    for pesagem in pesagens:
        if pesagem['CodigoConteinerEntrada'] or pesagem['CodigoConteinerSaida']:
            pesagem_insert_mongo = {'codigo_recinto': recinto}
            pesagem_insert_mongo.update(trata_pesagem_dte(pesagem))
            # print(pesagem_insert_mongo)
            #break
            i += 1
print(i)

1004
